In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

%matplotlib inline

In [3]:
# Packages for serial communication and timing
import serial
import time


### Sending a command, and then just printing the output.

In [4]:

import time
from typing import Optional
import serial
from serial.tools import list_ports

PORT = "COM7"          
BAUD = 115200
TIMEOUT_S = 2.0

_ser: Optional[serial.Serial] = None

def ports():
    """List available serial ports."""
    return [f"{p.device} - {p.description}" for p in list_ports.comports()]

def open_serial():
    """Open port once per session."""
    global _ser
    if _ser and _ser.is_open:
        return _ser
    _ser = serial.Serial(PORT, baudrate=BAUD, timeout=TIMEOUT_S)
    time.sleep(2.0)              # allow Arduino auto-reset
    _ser.reset_input_buffer()
    _ser.reset_output_buffer()
    return _ser

def close_serial():
    global _ser
    if _ser:
        try: _ser.close()
        finally: _ser = None

def send_mm(dist_mm: float) -> Optional[float]:
    """
    Send distance in mm. Returns angle in degrees as float, or None on timeout/parse error.
    """
    s = open_serial()
    s.write(f"{dist_mm}\n".encode("ascii"))
    s.flush()
    line = s.readline().decode("ascii", errors="ignore").strip()
    if not line:
        return None
    try:
        return float(line)
    except ValueError:
        print(f"Raw response: {line!r}")
        return None


In [11]:
open_serial()
send_mm(-150.0)

SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))

### Processing a data dump

In [ ]:
# Plot the data
plt.figure(figsize=(10, 5))
plt.plot(df['time_ms'], df['sine_value'], marker='o', linestyle='-')
plt.title('Sine Wave Data from Arduino')
plt.xlabel('Time (ms)')
plt.ylabel('Sine Value')
plt.grid()
plt.show()  

# Save the file as a CSV
df.to_csv('sine_wave_data.csv', index=False)

### Sample experiment with multiple runs and data collection

In [ ]:
def collect_and_print_serial_output(arduino, duration_s):    
    # Collect serial output from the Arduino for 1 second, or until 'Done' message is sent.
    # Return an array of all the strings sent through serial 
    start_time = time.time()
    data = []
    while (time.time() - start_time) < (duration_s + 2):  # Give some extra time for processing
        if arduino.in_waiting > 0:
            line = arduino.readline().decode('utf-8').rstrip()
            # If the line starts with 'Done', we can stop reading
            if line.startswith("Done"):
                print(line)
                break        
            data.append(line)
            print(line)
    return data

In [ ]:
df_by_frequency = {}
for frequency in [1.7, 2.3, 3.0]:
    # Use a high baud rate for faster data transfer
    arduino = serial.Serial(port=com_port, baudrate=115200, timeout=.1)
    time.sleep(2)  # Give some time for the connection to establish

    # Send the sine wave generation command to the Arduino
    # Format: S <frequency_Hz>,<duration in s>\n
    frequency_Hz = frequency  # Frequency of the sine wave in Hz
    duration_s = 5.0   # Duration to generate the sine wave in seconds
    command = f"S {frequency_Hz},{int(duration_s)}\n"
    print(f"Sending command: {command.strip()}")
    arduino.write(command.encode('utf-8'))
    # Wait for the Arduino to process the command
    time.sleep(1)
    # Collect serial output from the Arduino for 1 second, or until 'Done' message is sent.
    # Just print it to the console for now.
    data = collect_and_print_serial_output(arduino, 3)

    # Send the data dump command to the Arduino
    command = "D"
    print(f"Sending command: {command.strip()}")
    arduino.write(command.encode('utf-8'))
    # Return values are CSV strings of the form:
    # <timestamp_ms>,sine_value
    # Collect serial output from the Arduino for 10 seconds, or until 'Done' message is sent.
    data = collect_and_print_serial_output(arduino, 10)
    # Convert the data to a DataFrame.  Store it in a list or dictionary if you want to keep multiple runs.
    df = pd.DataFrame([x.split(',') for x in data], columns=['time_ms', 'sine_value'])
    df['time_ms'] = pd.to_numeric(df['time_ms'], errors='coerce')
    df['sine_value'] = pd.to_numeric(df['sine_value'], errors='coerce')
    # Store in the df_by_frequency dictionary
    df_by_frequency[frequency] = df
    # Close the serial connection
    arduino.close()

# Plot all the data on a single graph
plt.figure(figsize=(10, 5))
for freq, df in df_by_frequency.items():
    plt.plot(df['time_ms'], df['sine_value'], marker='o', linestyle='-',  label=f'{freq} Hz')
    plt.title('Sine Wave Data from Arduino at Different Frequencies')
    plt.xlabel('Time (ms)')
    plt.ylabel('Sine Value')
    plt.grid()
    plt.legend()
plt.show()